# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

import os

# Import API key
from api_key import APIkey1

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#get data from Part I
#DATOS = '../Weatherpy/WeaterBycity.csv'

#make DataFrame
data_df = pd.read_csv('../Weatherpy/WeaterBycity.csv')
data = data_df.drop(columns=['Unnamed: 0'])

data.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        563 non-null    object 
 1   Country     558 non-null    object 
 2   Lat         563 non-null    float64
 3   Lng         563 non-null    float64
 4   Cloudiness  563 non-null    int64  
 5   Date        563 non-null    int64  
 6   Humidity    563 non-null    int64  
 7   Max Temp    563 non-null    float64
 8   Wind Speed  563 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 35.2+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key = APIkey1)

In [4]:
#set variables for gmaps
humidity = data['Humidity'].astype(float)
Max_humidity = humidity.max()
loc = data[["Lat","Lng"]]
fig = gmaps.figure()
heat_layer =  gmaps.heatmap_layer(loc,weights=humidity,dissipating=False, max_intensity=Max_humidity,point_radius=3)
#Add Heatmap layer to map.
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions.(70<temp max<80,Wind speed <10, cloudiness ==0

ND_data = data.loc[(data["Wind Speed"] <= 10) & (data["Cloudiness"]== 0) & \
                  (data["Max Temp"] >= 294.261
                  ) & (data["Max Temp"] <= 299.817)].dropna()
ND_data.head()



,City,Country,Lat,Lng,Cloudiness,Date,Humidity,Max Temp,Wind Speed
49,Balkhash,KZ,46.8440,74.9804,0,1619589067,19,297.15,2.00
65,Ipixuna,BR,-1.7625,-48.8043,0,1619589072,94,298.15,2.06
149,Xianshuigu,CN,38.9850,117.3828,0,1619589099,15,295.15,5.00
323,Morondava,MG,-20.2833,44.2833,0,1619589156,73,299.79,2.88
374,Vao,NC,-22.6667,167.4833,0,1619589173,60,296.18,7.85


In [6]:
ND_data.reset_index(inplace=True)
del ND_data['index']
ND_data.head()

,City,Country,Lat,Lng,Cloudiness,Date,Humidity,Max Temp,Wind Speed
0,Balkhash,KZ,46.8440,74.9804,0,1619589067,19,297.15,2.00
1,Ipixuna,BR,-1.7625,-48.8043,0,1619589072,94,298.15,2.06
2,Xianshuigu,CN,38.9850,117.3828,0,1619589099,15,295.15,5.00
3,Morondava,MG,-20.2833,44.2833,0,1619589156,73,299.79,2.88
4,Vao,NC,-22.6667,167.4833,0,1619589173,60,296.18,7.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:

#variable for storing hotels names
Hotels5K = []
#loop throght lat, lng, city
for i in range(len(ND_data)):
    lat = ND_data.loc[i]['Lat']
    lng = ND_data.loc[i]['Lng']
    city= ND_data.loc[i]['City']
#Set parameters to search for hotels with 5000 meters.    
    conditionals = {
        'location':f'{lat},{lng}',
        'radius': 5000,
        'types': 'hotel',
        'key': APIkey1
    }
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=conditionals)
    jsn = requested.json()
    try:
#Store the first Hotel result into the DataFrame.
        
        Hotels5K.append(jsn['results'][0]['name'])
    except:
#Add a "Hotel Name" column to the DataFrame

        Hotels5K.append("")
ND_data["Hotel Name"] = Hotels5K
ND_data = ND_data.dropna(how='any')
hotel_df= ND_data
hotel_df



,City,Country,Lat,Lng,Cloudiness,Date,Humidity,Max Temp,Wind Speed,Hotel Name
0,Balkhash,KZ,46.8440,74.9804,0,1619589067,19,297.15,2.00,Balkhash
1,Ipixuna,BR,-1.7625,-48.8043,0,1619589072,94,298.15,2.06,Colônia Velha
2,Xianshuigu,CN,38.9850,117.3828,0,1619589099,15,295.15,5.00,Tianjin
3,Morondava,MG,-20.2833,44.2833,0,1619589156,73,299.79,2.88,Morondava
4,Vao,NC,-22.6667,167.4833,0,1619589173,60,296.18,7.85,Vao
5,Kashgar,CN,39.4547,75.9797,0,1619589175,13,296.15,2.00,Kashgar Prefecture
6,Acapulco de Juárez,MX,16.8634,-99.8901,0,1619589201,73,299.14,1.14,Acapulco
7,Qingdao,CN,36.0986,120.3719,0,1619589206,24,295.15,4.00,Qingdao
8,Maţāy,EG,28.4190,30.7792,0,1619589213,38,295.02,5.00,Matay
9,Gibara,CU,21.1097,-76.1317,0,1619589224,88,296.15,3.60,Gibara


In [9]:
hotel_df.to_csv('hotel_export.csv')

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))